In [63]:
# Imports and definitions
import numpy as np
from collections import defaultdict
from collections import namedtuple
import urllib.request
import hashlib
import Crypto
from Crypto.PublicKey import RSA
from Crypto.Hash import SHA256, SHA, SHA512, MD5
from Crypto import Random
import base64

In [153]:
HashPointer = namedtuple('HashPointer', ['hash', 'pointer'])
Transaction = namedtuple('Transaction', ['payer', 'payee', 'amount'])
Puzzle = namedtuple('Puzzle', ['difficulty', 'denominator', 'numerator'])
User = namedtuple('User', ['secret_key', 'public_key']) 

class Block:
    def __init__(self, transaction, signature, prev, nonce):
        self.transaction = transaction
        self.signature = signature
        self.prev = prev
        self.nonce = nonce
    
    def __repr__(self):
        return f'\nBlock(\n transaction: {self.transaction},\n signature: {self.signature},\n nonce: {self.nonce},\n prev: {self.prev})' 
    
class Blockchain:
    def __init__(self):
        self.users = {}
        self.length = 0
        self.blockchain = None
    
    def get_puzzle(self):
        return self.puzzle
        
    def build_genesis(self, transaction, signature):
        if self.length == 0:
            isValid, errors = self.process_transaction(transaction, signature)

            if isValid:
                self.blockchain = Block(transaction, signature, None, None)
                self.puzzle = self.new_puzzle()
                self.length = 1
                return True
            else:
                print(errors)
                return False
        else:
            print('Genesis already exists.')
            return False
        
    def add_transaction(self, transaction, signature, nonce):
        isValid, errors = self.process_transaction(transaction, signature)
        
        if isValid:
            self.blockchain = Block(transaction, signature, self.blockchain, nonce)
            self.length  = self.length + 1
            return True
        else:
            print(errors)
            return False
        
    def new_user(self):  
        length=1024  
        secret_key = RSA.generate(length, Random.new().read)  
        public_key = secret_key.publickey()
        user = User(secret_key, public_key)
        return user
        
    def sign(self, sk, transaction):
        hashed_transaction = self.hash_object(transaction)
        return base64.b64encode(str((sk.sign(hashed_transaction,''))[0]).encode())
    
    def verify_signature(self, pk, sig, transaction):
        if pk is None:
            return True
        else:
            return pk.verify(self.hash_object(transaction),(int(base64.b64decode(sig)),))
    
    def process_transaction(self, transaction, signature):
        users = self.users
        payer = transaction.payer
        payer_hash = self.hash_object(payer)
        payee = transaction.payee
        payee_hash = self.hash_object(payee)
        amount = transaction.amount
        isValid = True
        error = ''
        
        if not self.verify_signature(payer, signature, transaction):
            isValid = False
            error += '\n Invalid signature.'
        
        if payer is None and payee is not None:
            if payee_hash not in users.keys():
                users[payee_hash] = amount
            else:
                isValid = False
                error += '\n This user already exists in the system.'
        elif payer is not None and payee is not None:
            if payee_hash in users.keys() and payer_hash in users.keys():
                payer_balance = users[payer_hash]
                payee_balance = users[payee_hash]
                
                if payer_balance - amount < 0:
                    isValid = False
                    error += '\n Payer does not have enough currency. '
                else:
                    users[payer_hash] = payer_balance - amount
                    users[payee_hash] = payee_balance + amount
                    
            else:
                if payee_hash not in users.keys():
                    isValid = False
                    error += '\n Payee does not exist. '
                if payer_hash not in users.keys():
                    isValid = False
                    error += '\n Payer does not exist. '
        elif payee is None:
            isValid = False
            error += '\n Payee not specified. '   
                
        self.users = users
        return isValid, error
    
    def new_puzzle(self):
        try:
            self.puzzle
        except AttributeError:
            denominator = 1  
        else:
            denominator = self.puzzle.denominator     
        
        hash_string = hashlib.sha256(bytes('hello', encoding='utf-8')).digest()
        denominator = np.random.randint(denominator, denominator*17)
        numerator = 2**(len(hash_string) * 8)
        difficulty = int(numerator/denominator)
        return Puzzle(difficulty, denominator, numerator)
    
    def hash_object(self, obj):
        return int.from_bytes(hashlib.sha256(bytes(str(obj), encoding='utf-8')).digest(), 'big')
        

In [160]:
## Create transaction, create signature of that transaction, and use it to create the blockchain.
bc = Blockchain()

u1 = new_user()

t1 = Transaction(None, u1.public_key, 1000)
s1 = bc.sign(u1.secret_key, t1)

assert bc.blockchain == None
assert bc.length == 0
print(bc.blockchain)
bc.build_genesis(t1, s1)
print(bc.blockchain)
assert bc.length == 1

## Add a transaction to the above blockchain
u2 = bc.new_user()
t2 = Transaction(None, u2.public_key, 3500)
s2 = bc.sign(u2.secret_key, t2)

bc.add_transaction(t2, s2, None)
assert bc.length == 2
assert bc.users[bc.hash_object(u2.public_key)] == 3500 
print(bc.blockchain)

## Add a transaction that has one user pay another user
print(bc.users)
t3 = Transaction(u2.public_key, u1.public_key, 500)
s3 = bc.sign(u2.secret_key, t3)

bc.add_transaction(t3, s3, None)
assert bc.length == 3
assert bc.users[bc.hash_object(u1.public_key)] == 500 
print(bc.users)
print(bc.blockchain)

## Try to pay from an account that does not yet exist in the system
u3 = bc.new_user()
t4 = Transaction(u3.public_key, u2.public_key, 100)
s4 = bc.sign(u3.secret_key, t4)

bc.add_transaction(t4, s4, None)
assert bc.length == 3

## Try to pay to an account that does not yet exist in the system
u4 = bc.new_user()
t5 = Transaction(u2.public_key, u4.public_key, 100)
s5 = bc.sign(u4.secret_key, t5)

print(bc.users)
bc.add_transaction(t5, s5, None)
assert bc.length == 3
print(bc.users)

None

Block(
 transaction: Transaction(payer=None, payee=<_RSAobj @0x10f97ee20 n(1024),e>, amount=1000),
 signature: b'NjA0NDI2ODAzODc1MjMwOTE3NjQ3MDcyNjgwNTgzMDQxNzg3ODQ4MzEzMDQ0ODY4OTMzMzk3NDkxNDI0NTU3MjAwNzk3ODc0MzAyMTc4OTE4OTA2ODUzMzEzMjgzOTIzMDc4NDU4NzQwMDYyODUxMzc2NDc0NDgzMjI1NjExNTI5MjM5NjcyOTQwMDgwNjg0MzAzNzM1NTUzODEwNDg4MzI0NTY4Mjk2NDA1MTkyMjI5MjExNjgwMDkzNzYxOTY3OTE4MTQyNzgwNTE0NjI4MjczNzE0OTAwNTE5ODg2MjQ3OTM0NTM5ODEzMDExODk1MzE2Mzc1MzkzMDQ3OTc1MTY0MDc4NzY3Njg1MTMwNjM0MDkwMDAyNjcwNDY3Mjk2NTYxMjUwNjA4NjcyOTE3NDk3NzA=',
 nonce: None,
 prev: None)
3500

Block(
 transaction: Transaction(payer=None, payee=<_RSAobj @0x10f8eefa0 n(1024),e>, amount=3500),
 signature: b'NjM1MTU1MDE2MTI4NDgwNjYzODkxNjkxNDA4MjkxMjE2OTQ3OTQ5MzEzMDE4ODI4MzA1MjgyNTUyOTg5MTAzMTQ1NjUzODUyOTM5Mzg3Mjg5MTIwNDE5NDk2NjM2MjU5NjczMjkyOTkzMTE3NTI3NTI5ODA0NzUwNDYyNzk0MzQzNjUwOTA0MjIxMjgzMzIxMDg5OTk0MTc0NjMyNzMxNjI3MjUxMjk5Mzk4NTM0NDU3NDkzNDgzOTUzMjE5MjYzNTc5Njc5ODM0NzUwOTc2NzAyMTg1NDUzOTI4MTIzNjk1MTk5OTIyMjAyMDI0MjU4